In [33]:
import pandas as pd
import pyodbc
import yaml

## Load config file

with open("config.yaml", "r") as f:
    config = yaml.load(f, yaml.FullLoader)

## Organize details in dictionary

connect_dict = config["Connection details"]

assert len(pyodbc.drivers()) > 0, "Please install an ODBC driver"
connect_dict['DRIVER'] = pyodbc.drivers()[-1]

## Make connection string

conn_string = ";".join([f"{key}={value}" for key, value in connect_dict.items()])

## Connect to database

connection = pyodbc.connect(conn_string)

## Instruction was given to write and SQL query to extract:
# data between dates 2011 and 2012
# all columns EXCEPT guest-bike and registered-bike

col_names_substring = ",".join(config['Columns to extract'])
date_limits = config['Date limits']
query_string =  f"SELECT {col_names_substring} FROM rental_data WHERE date BETWEEN '{date_limits['first']}' and '{date_limits['last']}'"

df = pd.read_sql(sql = query_string,
                 con = connection,
                 parse_dates = "date")\
       .sort_values(["date", "hr"])\
       .assign(weather=lambda x: x.weather.str.lower())